# Motion and Optical Flow

In [ ]:
import numpy as np

from itertools import product

## Convolution Functions

In [ ]:
def pixel_convolution(image, mask, image_location):
    image_row, image_col = image_location
    mask_height, mask_width = mask.shape

    height_margin = mask_height // 2
    width_margin = mask_width // 2

    convolution_sum = 0

    for i, row in enumerate(range(-height_margin, height_margin + 1)):
        for j, col in enumerate(range(-width_margin, width_margin + 1)):
            convolution_sum += image[row + image_row, col + image_col] * mask[i, j]

    return convolution_sum

def image_convolution(image, mask):
    filtered_image = []

    mask_height, mask_width = mask.shape
    image_height, image_width = image.shape
    
    start_row = mask_height // 2
    start_col = mask_width // 2

    end_row = image_height - start_row
    end_col = image_width - start_col

    filtered_image = np.zeros((end_col - start_col, end_row - start_row))

    for i, row in enumerate(range(start_row, end_row)):
        for j, col in enumerate(range(start_col, end_col)):
            filtered_image[i, j] = pixel_convolution(image, mask, (row, col))
        
    return filtered_image

def add_padding(image, kernel_size):
    kernel_height, kernel_width = kernel_size
    image_height, image_width = image.shape

    if kernel_height % 2 == 0 or kernel_width % 2 == 0:
        raise ValueError("Kernel size must be odd-sized in both dimensions")

    padding_height = kernel_height // 2
    padding_width = kernel_width // 2

    padded_image_height = image_height + 2 * padding_height
    padded_image_width = image_width + 2 * padding_width

    padded_image = np.zeros((padded_image_height, padded_image_width))
    padded_image[
        padding_height : padded_image_height - padding_height,
        padding_width : padded_image_width - padding_width
    ] = image

    return padded_image
    

## Kernels

In [ ]:
pixel_gradient_kernel = np.array(
    [-1/12, 2/3, 0, -2/3, 1/12]
)

time_kernel = np.array(
    [-1, 1]
)

gaussian_kernel = np.array([
    [1, 4, 7, 4, 1],
    [4, 16, 26, 16, 4],
    [7, 24, 41, 26, 7],
    [4, 16, 26, 16, 4],
    [1, 4, 7, 4, 1]
]) / 273

## Main Script

In [ ]:
def lucas_kanade(frame_x, frame_y, region_size=25):
    if frame_x.shape != frame_y.shape:
        raise ValueError("Frames not the same shape.")

    frame_x_padded = add_padding(frame_x, (5, 1))
    frame_y_padded = add_padding(frame_y, (5, 1))
    frame_t_padded = add_padding(frame_x - frame_y, (2, 1))

    # Compute the derivatives
    x_derivative = image_convolution(frame_x_padded, pixel_gradient_kernel)
    y_derivative = image_convolution(frame_y_padded, pixel_gradient_kernel)
    t_derivative = image_convolution(frame_t_padded, time_kernel)

    # Smooth all the gradients
    x_derivative = image_convolution(x_derivative, gaussian_kernel)
    y_derivative = image_convolution(y_derivative, gaussian_kernel)
    t_derivative = image_convolution(t_derivative, gaussian_kernel)

    solution_image = np.empty(frame_x.shape)
    regions = range(region_size)

    for region in regions:
        indices = (region * region_size, (region + 1) * region_size)
        
        x_submatrix = x_derivative[indices]
        y_submatrix = y_derivative[indices]
        t_submatrix = t_derivative[indices]

        for (x, y, t) in product(regions, regions, regions):
            




